In [ ]:
from bs4 import BeautifulSoup
import requests
import sqlite3
import json
import csv

# Task 1: Scrape Weather Information
def scrape_weather():
    with open("weather.html", "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")
    
    weather_data = []
    rows = soup.find("table").find("tbody").find_all("tr")
    
    for row in rows:
        cols = row.find_all("td")
        day, temp, condition = cols[0].text, int(cols[1].text[:-2]), cols[2].text
        weather_data.append({"day": day, "temperature": temp, "condition": condition})
    
    print("Weather Forecast:")
    for entry in weather_data:
        print(entry)
    
    highest_temp = max(weather_data, key=lambda x: x["temperature"])
    print("Hottest Day:", highest_temp["day"], highest_temp["temperature"], "°C")
    
    sunny_days = [entry["day"] for entry in weather_data if entry["condition"] == "Sunny"]
    print("Sunny Days:", sunny_days)
    
    avg_temp = sum(entry["temperature"] for entry in weather_data) / len(weather_data)
    print("Average Temperature:", avg_temp, "°C")

# Task 2: Scrape Job Listings
def scrape_jobs():
    url = "https://realpython.github.io/fake-jobs"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    conn = sqlite3.connect("jobs.db")
    cursor = conn.cursor()
    
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS jobs (
        job_title TEXT,
        company_name TEXT,
        location TEXT,
        job_description TEXT,
        application_link TEXT,
        UNIQUE(job_title, company_name, location)
    )
    """)
    
    job_listings = []
    jobs = soup.find_all("div", class_="card-content")
    
    for job in jobs:
        title = job.find("h2", class_="title").text.strip()
        company = job.find("h3", class_="company").text.strip()
        location = job.find("p", class_="location").text.strip()
        description = job.find("div", class_="content").text.strip()
        apply_link = job.find("a", class_="apply")["href"]
        
        job_listings.append((title, company, location, description, apply_link))
    
    for job in job_listings:
        cursor.execute("""
        INSERT INTO jobs (job_title, company_name, location, job_description, application_link)
        VALUES (?, ?, ?, ?, ?)
        ON CONFLICT(job_title, company_name, location) DO UPDATE
        SET job_description = excluded.job_description, application_link = excluded.application_link
        """, job)
    
    conn.commit()
    conn.close()
    print("Job data scraped and stored.")

# Export filtered job data to CSV
def export_jobs_to_csv(filter_by, value):
    conn = sqlite3.connect("jobs.db")
    cursor = conn.cursor()
    
    cursor.execute(f"SELECT * FROM jobs WHERE {filter_by} = ?", (value,))
    jobs = cursor.fetchall()
    
    with open("filtered_jobs.csv", "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Job Title", "Company Name", "Location", "Description", "Application Link"])
        writer.writerows(jobs)
    
    conn.close()
    print(f"Filtered job data saved to filtered_jobs.csv.")

# Task 3: Scrape Laptop Data
def scrape_laptops():
    url = "https://www.demoblaze.com"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    laptops = []
    items = soup.find_all("div", class_="card-block")
    
    for item in items:
        name = item.find("h4", class_="card-title").text.strip()
        price = item.find("h5").text.strip()
        description = item.find("p", class_="card-text").text.strip()
        laptops.append({"name": name, "price": price, "description": description})
    
    with open("laptops.json", "w", encoding="utf-8") as file:
        json.dump(laptops, file, indent=4)
    
    print("Laptop data scraped and saved to laptops.json.")

# Run tasks
scrape_weather()
scrape_jobs()
scrape_laptops()